In [10]:
%pip install pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
from pathlib import Path

# Carpeta donde están tus archivos
carpeta = Path("Bases_Datos")

# Archivos que quieres leer
archivos = [
    carpeta / "DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx",
    carpeta / "DATOS HISTÓRICOS 2021_Contaminante.xlsx",
    carpeta / "DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx",
    carpeta / "DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx",
    carpeta / "DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx",
    carpeta / "DATOS HISTÓRICOS 2025_TODAS ESTACIONES.xlsx",
]

# Probar que se leen
for f in archivos:
    xl = pd.ExcelFile(f, engine="openpyxl")
    print(f"\n📂 {f.name}")
    print("Hojas:", xl.sheet_names)


📂 DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx
Hojas: ['SURESTE', 'NORESTE', 'CENTRO', 'NOROESTE', 'SUROESTE', 'NOROESTE2', 'NORTE', 'SUROESTE2', 'SURESTE2', 'SURESTE3', 'SUR', 'NORTE2', 'NORESTE2', 'NORESTE3', 'NOROESTE3', 'CATÁLOGO']

📂 DATOS HISTÓRICOS 2021_Contaminante.xlsx
Hojas: ['Param_horarios_Estaciones']

📂 DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx
Hojas: ['SURESTE', 'NORESTE', 'CENTRO', 'NOROESTE', 'SUROESTE', 'NOROESTE2', 'NORTE', 'SUROESTE2', 'SURESTE2', 'SURESTE3', 'SUR', 'NORTE2', 'NORESTE2', 'NORESTE3', 'NOROESTE3', 'CATÁLOGO']

📂 DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx
Hojas: ['Param_horarios_Estaciones', 'Hoja2']

📂 DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx
Hojas: ['SE', 'CE', 'SO', 'NE2', 'SE2', 'SE3', 'NE', 'NO', 'NO2', 'NTE', 'NTE2', 'SO2', 'SUR', 'NO3', 'NE3']

📂 DATOS HISTÓRICOS 2025_TODAS ESTACIONES.xlsx
Hojas: ['Variables', 'Observaciones']


In [13]:
# --- BLOQUE ROBUSTO PARA EXTRAER NOROESTE / NOROESTE2 / NOROESTE3 ---
# Funciona con archivos dentro de "Bases_Datos"
# y guarda "Datos_Noroeste.csv" en la raíz del proyecto.

%pip install -q pandas openpyxl

from pathlib import Path
import pandas as pd
import re
import unicodedata

# 0) Parámetros
CARPETA = Path("Bases_Datos")     # si tu carpeta cambia, ajusta aquí
SALIDA  = Path("Datos_Noroeste.csv")

# 1) Verifica dónde estás y que existan los archivos
print("📂 Working dir:", Path.cwd())
if not CARPETA.exists():
    raise FileNotFoundError(f"No existe la carpeta '{CARPETA}'. Revisa tu estructura.")

archivos = sorted(CARPETA.glob("*.xlsx"))
if not archivos:
    raise FileNotFoundError(f"No se encontraron .xlsx dentro de '{CARPETA}'.")
print("Archivos encontrados:", [a.name for a in archivos])

# 2) Diagnóstico: imprime hojas para confirmar nombres
print("\n🔎 Hojas por archivo:")
for f in archivos:
    try:
        xl = pd.ExcelFile(f, engine="openpyxl")
        print(f" - {f.name}: {xl.sheet_names}")
    except Exception as e:
        print(f" - {f.name}: ERROR al abrir -> {e}")

# 3) Normalizador y patrón de coincidencia
def norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = s.upper().strip()
    s = re.sub(r"[\s\-\_\.]+", "", s)  # quita espacios/guiones/guiones bajos/puntos
    return s

# Queremos: NOROESTE, NOROESTE2, NOROESTE3 (alias: NO, NO2, NO3)
# Permitimos variantes como "NO 2", "NO-3", "NOROESTE 3", etc.
def a_estacion_canonica(nombre_hoja: str):
    s = norm(nombre_hoja)
    # patrones para NO, NO2, NO3
    if re.fullmatch(r"NO", s):   return "NOROESTE"
    if re.fullmatch(r"NO2", s):  return "NOROESTE2"
    if re.fullmatch(r"NO3", s):  return "NOROESTE3"
    # patrones para NOROESTE, NOROESTE2, NOROESTE3
    if re.fullmatch(r"NOROESTE", s):   return "NOROESTE"
    if re.fullmatch(r"NOROESTE2", s):  return "NOROESTE2"
    if re.fullmatch(r"NOROESTE3", s):  return "NOROESTE3"
    return None

# 4) Extracción con logging
noroeste_frames = []
log_match = []
log_omit  = []

for f in archivos:
    try:
        xl = pd.ExcelFile(f, engine="openpyxl")
        for sheet in xl.sheet_names:
            canon = a_estacion_canonica(sheet)
            if canon is not None:
                try:
                    df = xl.parse(sheet)
                    df["Fuente"]   = f.name
                    df["Estacion"] = canon
                    noroeste_frames.append(df)
                    log_match.append((f.name, sheet, canon, len(df)))
                except Exception as e_parse:
                    log_omit.append((f.name, sheet, f"ERROR al leer: {e_parse}"))
            else:
                log_omit.append((f.name, sheet, "omitida"))
    except Exception as e_open:
        log_omit.append((f.name, "(archivo)", f"ERROR al abrir: {e_open}"))

# 5) Validación y guardado
if not noroeste_frames:
    # Ayuda para depurar si no encontró nada
    print("\n⚠️ No se encontraron hojas coincidentes (NOROESTE/2/3 o NO/2/3).")
    print("Mira el diagnóstico arriba y ajusta el matcher si tus hojas tienen otro nombre.")
    print("\nEjemplos de hojas omitidas (primeras 30):")
    for a, h, nota in log_omit[:30]:
        print(f"  - {a} | {h} | {nota}")
    raise ValueError("No objects to concatenate (no hubo coincidencias).")

combined = pd.concat(noroeste_frames, ignore_index=True)
combined.to_csv(SALIDA, index=False, encoding="utf-8-sig")

print("\n✅ CSV creado:", SALIDA.resolve())
print("📄 Coincidencias:")
for a, h, est, n in log_match:
    print(f"  - {a} | hoja '{h}' -> {est} | filas={n}")

print("\nResumen por Estacion:")
print(combined["Estacion"].value_counts(dropna=False))

Note: you may need to restart the kernel to use updated packages.
📂 Working dir: /Users/brismaalvarezvaldez/Documents/GitHub/SIMA_Project_2
Archivos encontrados: ['DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx', 'DATOS HISTÓRICOS 2021_Contaminante.xlsx', 'DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx', 'DATOS HISTÓRICOS 2023_2024_TODAS ESTACIONES_ITESM.xlsx', 'DATOS HISTÓRICOS 2024_TODAS ESTACIONES.xlsx', 'DATOS HISTÓRICOS 2025_TODAS ESTACIONES.xlsx']

🔎 Hojas por archivo:
 - DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx: ['SURESTE', 'NORESTE', 'CENTRO', 'NOROESTE', 'SUROESTE', 'NOROESTE2', 'NORTE', 'SUROESTE2', 'SURESTE2', 'SURESTE3', 'SUR', 'NORTE2', 'NORESTE2', 'NORESTE3', 'NOROESTE3', 'CATÁLOGO']
 - DATOS HISTÓRICOS 2021_Contaminante.xlsx: ['Param_horarios_Estaciones']
 - DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx: ['SURESTE', 'NORESTE', 'CENTRO', 'NOROESTE', 'SUROESTE', 'NOROESTE2', 'NORTE', 'SUROESTE2', 'SURESTE2', 'SURESTE3', 'SUR', 'NORTE2', 'NORESTE2', 'NORESTE3

In [ ]:
# Suponiendo que ya todo lo tenemos en un data frame guardado

import pandas as pd 
import numpy as numpy
import matplotlib.pyplot as plt

In [ ]:
df = # Poner lo del data frame 
print ("Dimensión del data set:", df.shape)

In [ ]:
print("\nTipos de datos:")
print (df.types)

In [ ]:
print("\nPrimeras filas:")
display(df.head())

In [ ]:
print("\nValores nulos por columna:")
nulls = df.isna().sum().sort_values(ascending=False)
display(nulls.to_frame("n_nulos"))

In [ ]:
print("\nPorcentaje de nulos por columna:")
display((df.isna().mean()*100).round(2).sort_values(ascending=False).to_frame("%_nulos"))

In [ ]:
display(df.describe())

In [ ]:
obj_cols = df.select_dtypes(exclude="number").columns.tolist()
if obj_cols:
    print("\nEstadísticas descriptivas (categóricas):")
    display(df[obj_cols].describe())

In [ ]:
for col in obj_cols:
    print(f"\nTop categorías en {col}:")
    display(df[col].value_counts(dropna=False).head(10)

In [ ]:
dup_count = df.duplicated().sum()
print(f"\nFilas duplicadas: {dup_count}")

In [ ]:
num_cols = df.select_dtypes(include="number").columns.tolist()

In [ ]:
for col in num_cols[:12]: 
    plt.figure()
    plt.hist(df[col].dropna(), bins=30)
    plt.title(f"Histograma: {col}")
    plt.xlabel(col); plt.ylabel("Frecuencia")
    plt.show()

In [ ]:
for col in num_cols[:12]:
    plt.figure()
    plt.boxplot(df[col].dropna(), vert=False)
    plt.title(f"Boxplot: {col}")
    plt.xlabel(col)
    plt.show()

In [ ]:
plt.imshow(df.isna(), aspect='auto', interpolation='nearest')
plt.title("Mapa simple de valores nulos")
plt.xlabel("Columnas"); plt.ylabel("Filas")
plt.colorbar(label="Nulo=1 / No nulo=0")
plt.show()

In [ ]:
# Falta la preparacion y transformacion (IMPUTACION)